In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import csv
from tsfresh.feature_extraction import extract_features, MinimalFCParameters
%matplotlib inline

WINDOW = 5000
FILE = r'train.csv'


In [18]:
i = 0
target = []
mydata = pd.DataFrame()
for chunk in pd.read_csv(FILE, chunksize=WINDOW,dtype={'acoustic_data': np.int8, 'time_to_failure': np.float32 },nrows=100000):
    i += 1
    chunk['id'] = i
    extracted_features = extract_features(chunk.drop(columns=['time_to_failure']), column_id='id', n_jobs=4,default_fc_parameters=MinimalFCParameters())
    mydata = pd.concat([mydata, extracted_features])
    target.append(chunk.time_to_failure.iloc[-1])

mydata['time_to_failure'] = target
mydata.to_csv(r'ft_1m5.csv', index=False)

Feature Extraction: 100%|██████████| 1/1 [00:00<00:00, 459.15it/s]


In [21]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

In [22]:
dataframe = pd.read_csv(r'ft_1m5.csv')
dataframe = dataframe.dropna(axis=1)
dataframe.shape

(20, 9)

In [23]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataframe.drop(columns=['time_to_failure']), dataframe.time_to_failure, test_size=0.3)

In [24]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [25]:
def create_model(optimizer='adam'):
    model = Sequential()
    model.add(Dense(64,
                    input_dim=X_train.shape[1],
                    #kernel_initializer='normal',
                    activation='relu'                   
                   ))
    model.add(Dense(1,
                   #kernel_initializer='normal',
                   activation='linear'
                   ))
    model.compile(loss='mse', optimizer=optimizer)
    return model

In [26]:
model = KerasRegressor(build_fn=create_model, epochs=50, batch_size=64, verbose=1, shuffle=True)

In [27]:
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, verbose=1)
grid_result = grid.fit(X_train_scaled, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Fitting 3 folds for each of 7 candidates, totalling 21 fits


/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:    5.0s finished
/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


Epoch 1/50
14/14 [==============================] - 0s 5ms/step - loss: 2.1151
Epoch 2/50
14/14 [==============================] - 0s 49us/step - loss: 1.8030
Epoch 3/50
14/14 [==============================] - 0s 86us/step - loss: 1.5457
Epoch 4/50
14/14 [==============================] - 0s 52us/step - loss: 1.3265
Epoch 5/50
14/14 [==============================] - 0s 51us/step - loss: 1.1385
Epoch 6/50
14/14 [==============================] - 0s 50us/step - loss: 0.9793
Epoch 7/50
14/14 [==============================] - 0s 67us/step - loss: 0.8419
Epoch 8/50
14/14 [==============================] - 0s 57us/step - loss: 0.7251
Epoch 9/50
14/14 [==============================] - 0s 42us/step - loss: 0.6258
Epoch 10/50
14/14 [==============================] - 0s 46us/step - loss: 0.5413
Epoch 11/50
14/14 [==============================] - 0s 48us/step - loss: 0.4699
Epoch 12/50
14/14 [==============================] - 0s 82us/step - loss: 0.4089
Epoch 13/50
14/14 [===================

In [36]:
from sklearn import preprocessing
print(y_train)
lab_enc = preprocessing.LabelEncoder()
y_train_encoded = lab_enc.fit_transform(y_train)
print(y_train_encoded)

11    1.454197
5     1.461699
19    1.443598
6     1.460598
15    1.448898
9     1.456399
2     1.465897
1     1.466998
16    1.447897
12    1.453196
17    1.446796
10    1.455298
0     1.468099
13    1.451100
Name: time_to_failure, dtype: float64
[ 6 10  0  9  3  8 11 12  2  5  1  7 13  4]


In [31]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(X_train,y_train_encoded)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [32]:
predictions = LR.predict(X_test)
print(predictions)

[ 9  3 11 11  9  0]
